In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 활성화

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from PIL import Image
import math
import numpy as np
import time

# SRCNN 모델 정의
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 9, padding=0)  # 패딩 제거
        self.conv2 = nn.Conv2d(64, 32, 1, padding=0)
        self.conv3 = nn.Conv2d(32, 1, 5, padding=0)  # 패딩 제거

    def forward(self, x):
        x = F.pad(x, (4, 4, 4, 4), mode='replicate')  # 직접 패딩 추가
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.pad(x, (2, 2, 2, 2), mode='replicate')  # 직접 패딩 추가
        x = self.conv3(x)
        return x

# 가중치 초기화
def initialize_weights(model):
    classname = model.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)

model = SRCNN()
model.apply(initialize_weights)

# 손실 함수와 옵티마이저 설정
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# PSNR 함수 정의
def psnr(label, outputs, max_val=1.):
    label = label.cpu().detach().numpy()
    outputs = outputs.cpu().detach().numpy()
    img_diff = outputs - label
    rmse = math.sqrt(np.mean((img_diff)**2))
    if rmse == 0:
        return 100
    else:
        psnr = 20 * math.log10(max_val / rmse)
        return psnr

# CustomDataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("L")  # 흑백 이미지로 변환
        if self.transform:
            image = self.transform(image)
        return image, img_path  # 이미지 경로를 함께 반환

# 이미지 폴더 경로 설정
image_folder = './train'

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),
])

# 데이터셋 및 데이터 로더 생성
train_dataset = CustomDataset(image_folder=image_folder, transform=transform)
val_dataset = CustomDataset(image_folder=image_folder, transform=transform)

# 데이터셋의 크기가 충분한지 확인
if len(train_dataset) == 0 or len(val_dataset) == 0:
    raise ValueError("데이터셋이 비어 있습니다. 올바른 이미지 폴더를 설정하십시오.")

# 적절한 배치 크기 설정
batch_size = min(32, len(train_dataset), len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 학습 및 검증 함수 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def train(model, data_dl):
    model.train()
    running_loss = 0.0
    running_psnr = 0.0

    for ba, data in enumerate(data_dl):
        image = data[0].to(device)  # image를 GPU로 이동
        label = image  # label을 이미지 자체로 설정

        optimizer.zero_grad()
        outputs = model(image)
        loss = loss_func(outputs, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        batch_psnr = psnr(label, outputs)
        running_psnr += batch_psnr
    
    final_loss = running_loss / len(data_dl)
    final_psnr = running_psnr / len(data_dl)
    return final_loss, final_psnr

def validate(model, data_dl):
    model.eval()
    running_loss = 0.0
    running_psnr = 0.0
    with torch.no_grad():
        for ba, data in enumerate(data_dl):
            image = data[0].to(device)  # image를 GPU로 이동
            label = image  # label을 이미지 자체로 설정

            outputs = model(image)
            loss = loss_func(outputs, label)

            running_loss += loss.item()
            batch_psnr = psnr(label, outputs)
            running_psnr += batch_psnr

    final_loss = running_loss / len(data_dl)
    final_psnr = running_psnr / len(data_dl)
    return final_loss, final_psnr

def save_sr_images(model, data_dl):
    model.eval()
    with torch.no_grad():
        for ba, data in enumerate(data_dl):
            image = data[0].to(device)  # image를 GPU로 이동
            img_paths = data[1]

            outputs = model(image)
            outputs = outputs.cpu()
            for i, img_path in enumerate(img_paths):
                save_image(outputs[i], os.path.join('outputs', os.path.basename(img_path)))

# 출력 폴더 생성
if not os.path.exists('outputs'):
    os.mkdir('outputs')

# 학습 루프 실행
num_epochs = 100

train_loss, val_loss = [], []
train_psnr, val_psnr = [], []
start = time.time()
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1} of {num_epochs}')
    train_epoch_loss, train_epoch_psnr = train(model, train_loader)
    val_epoch_loss, val_epoch_psnr = validate(model, val_loader)

    train_loss.append(train_epoch_loss)
    train_psnr.append(train_epoch_psnr)
    val_loss.append(val_epoch_loss)
    val_psnr.append(val_epoch_psnr)
    end = time.time()
    print(f'Train PSNR: {train_epoch_psnr:.3f}, Val PSNR: {val_epoch_psnr:.3f}, Time: {end-start:.2f} sec')

# 최종 결과 저장
save_sr_images(model, val_loader)


Epoch 1 of 100
Train PSNR: 15.526, Val PSNR: 17.683, Time: 2.86 sec
Epoch 2 of 100
Train PSNR: 18.884, Val PSNR: 20.958, Time: 5.77 sec
Epoch 3 of 100
Train PSNR: 25.499, Val PSNR: 28.821, Time: 8.58 sec
Epoch 4 of 100
Train PSNR: 31.217, Val PSNR: 29.331, Time: 11.48 sec
Epoch 5 of 100
Train PSNR: 30.178, Val PSNR: 30.701, Time: 14.33 sec
Epoch 6 of 100
Train PSNR: 32.095, Val PSNR: 34.568, Time: 17.24 sec
Epoch 7 of 100
Train PSNR: 35.105, Val PSNR: 36.062, Time: 20.12 sec
Epoch 8 of 100
Train PSNR: 35.479, Val PSNR: 35.516, Time: 23.03 sec
Epoch 9 of 100
Train PSNR: 35.488, Val PSNR: 35.733, Time: 25.93 sec
Epoch 10 of 100
Train PSNR: 36.041, Val PSNR: 36.208, Time: 28.84 sec
Epoch 11 of 100
Train PSNR: 36.354, Val PSNR: 36.498, Time: 31.72 sec
Epoch 12 of 100
Train PSNR: 36.634, Val PSNR: 36.668, Time: 34.60 sec
Epoch 13 of 100
Train PSNR: 36.913, Val PSNR: 36.996, Time: 37.53 sec
Epoch 14 of 100
Train PSNR: 37.117, Val PSNR: 37.316, Time: 40.35 sec
Epoch 15 of 100
Train PSNR: 37.4

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 활성화

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from PIL import Image
import math
import numpy as np
import time

# SRCNN 모델 정의
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, 9, padding=0)  # 필터 수 증가
        self.conv2 = nn.Conv2d(128, 64, 1, padding=0)
        self.conv3 = nn.Conv2d(64, 1, 5, padding=0)  # 필터 수 증가

    def forward(self, x):
        x = F.pad(x, (4, 4, 4, 4), mode='replicate')  # 직접 패딩 추가
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.pad(x, (2, 2, 2, 2), mode='replicate')  # 직접 패딩 추가
        x = self.conv3(x)
        return x

# 가중치 초기화
def initialize_weights(model):
    classname = model.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal_(model.weight.data, nonlinearity='relu')  # Kaiming He 초기화 사용

model = SRCNN()
model.apply(initialize_weights)

# 손실 함수와 옵티마이저 설정
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # 학습률 조정
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)  # 학습률 감소 스케줄러

# PSNR 함수 정의
def psnr(label, outputs, max_val=1.):
    label = label.cpu().detach().numpy()
    outputs = outputs.cpu().detach().numpy()
    img_diff = outputs - label
    rmse = math.sqrt(np.mean((img_diff)**2))
    if rmse == 0:
        return 100
    else:
        psnr = 20 * math.log10(max_val / rmse)
        return psnr

# CustomDataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("L")  # 흑백 이미지로 변환
        if self.transform:
            image = self.transform(image)
        return image, img_path  # 이미지 경로를 함께 반환

# 이미지 폴더 경로 설정
image_folder = './train'

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 데이터 증강: 좌우 반전
    transforms.RandomVerticalFlip(),    # 데이터 증강: 상하 반전
    transforms.ToTensor(),
])

# 데이터셋 및 데이터 로더 생성
train_dataset = CustomDataset(image_folder=image_folder, transform=transform)
val_dataset = CustomDataset(image_folder=image_folder, transform=transform)

# 데이터셋의 크기가 충분한지 확인
if len(train_dataset) == 0 or len(val_dataset) == 0:
    raise ValueError("데이터셋이 비어 있습니다. 올바른 이미지 폴더를 설정하십시오.")

# 적절한 배치 크기 설정
batch_size = min(32, len(train_dataset), len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 학습 및 검증 함수 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def train(model, data_dl):
    model.train()
    running_loss = 0.0
    running_psnr = 0.0

    for ba, data in enumerate(data_dl):
        image = data[0].to(device)  # image를 GPU로 이동
        label = image  # label을 이미지 자체로 설정

        optimizer.zero_grad()
        outputs = model(image)
        loss = loss_func(outputs, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        batch_psnr = psnr(label, outputs)
        running_psnr += batch_psnr
    
    final_loss = running_loss / len(data_dl)
    final_psnr = running_psnr / len(data_dl)
    return final_loss, final_psnr

def validate(model, data_dl):
    model.eval()
    running_loss = 0.0
    running_psnr = 0.0
    with torch.no_grad():
        for ba, data in enumerate(data_dl):
            image = data[0].to(device)  # image를 GPU로 이동
            label = image  # label을 이미지 자체로 설정

            outputs = model(image)
            loss = loss_func(outputs, label)

            running_loss += loss.item()
            batch_psnr = psnr(label, outputs)
            running_psnr += batch_psnr

    final_loss = running_loss / len(data_dl)
    final_psnr = running_psnr / len(data_dl)
    return final_loss, final_psnr

def save_sr_images(model, data_dl):
    model.eval()
    with torch.no_grad():
        for ba, data in enumerate(data_dl):
            image = data[0].to(device)  # image를 GPU로 이동
            img_paths = data[1]

            outputs = model(image)
            outputs = outputs.cpu()
            for i, img_path in enumerate(img_paths):
                save_image(outputs[i], os.path.join('outputs', os.path.basename(img_path)))

# 출력 폴더 생성
if not os.path.exists('outputs'):
    os.mkdir('outputs')

# 학습 루프 실행
num_epochs = 100

train_loss, val_loss = [], []
train_psnr, val_psnr = [], []
start = time.time()
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1} of {num_epochs}')
    train_epoch_loss, train_epoch_psnr = train(model, train_loader)
    val_epoch_loss, val_epoch_psnr = validate(model, val_loader)

    train_loss.append(train_epoch_loss)
    train_psnr.append(train_epoch_psnr)
    val_loss.append(val_epoch_loss)
    val_psnr.append(val_epoch_psnr)

    scheduler.step()  # 학습률 감소

    end = time.time()
    print(f'Train PSNR: {train_epoch_psnr:.3f}, Val PSNR: {val_epoch_psnr:.3f}, Time: {end-start:.2f} sec')

# 최종 결과 저장
save_sr_images(model, val_loader)


Epoch 1 of 100


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Train PSNR: 26.290, Val PSNR: 27.389, Time: 6.75 sec
Epoch 2 of 100
Train PSNR: 29.717, Val PSNR: 31.595, Time: 10.57 sec
Epoch 3 of 100
Train PSNR: 30.760, Val PSNR: 31.905, Time: 14.40 sec
Epoch 4 of 100
Train PSNR: 33.337, Val PSNR: 33.198, Time: 18.22 sec
Epoch 5 of 100
Train PSNR: 33.708, Val PSNR: 34.569, Time: 22.05 sec
Epoch 6 of 100
Train PSNR: 34.764, Val PSNR: 35.409, Time: 25.89 sec
Epoch 7 of 100
Train PSNR: 35.641, Val PSNR: 35.731, Time: 29.74 sec
Epoch 8 of 100
Train PSNR: 36.213, Val PSNR: 36.554, Time: 33.62 sec
Epoch 9 of 100
Train PSNR: 36.780, Val PSNR: 37.072, Time: 37.50 sec
Epoch 10 of 100
Train PSNR: 37.234, Val PSNR: 37.520, Time: 41.33 sec
Epoch 11 of 100
Train PSNR: 37.672, Val PSNR: 37.890, Time: 45.18 sec
Epoch 12 of 100
Train PSNR: 38.084, Val PSNR: 38.213, Time: 49.02 sec
Epoch 13 of 100
Train PSNR: 38.329, Val PSNR: 38.497, Time: 52.91 sec
Epoch 14 of 100
Train PSNR: 38.647, Val PSNR: 38.738, Time: 56.78 sec
Epoch 15 of 100
Train PSNR: 38.894, Val PSNR: